In [6]:
import os
import openai

In [26]:
#os.environ['OPENAI_API_KEY'] = 'key'
openai.api_key = os.getenv('OPENAI_API_KEY')


In [11]:
prompt = "Give me details about the technology startup called Mimi and Pimo"

In [12]:
response = openai.Completion.create(engine = 'text-davinci-003',
                                    prompt = prompt,
                                    max_tokens=256,
                                    temperature = 0.7)

In [ ]:
print(response['choices'][0]['text'])



Mimi and Pimo is an early stage technology startup, founded by entrepreneurs and engineers, with a mission to revolutionize the way people manage their finances. The company's flagship product is an AI-powered financial assistant that enables users to take control of their finances and make informed decisions. The platform provides personalized financial advice based on individual data, including spending habits and income. It also helps users to track and manage their finances, set goals, create budgets, and analyze investments. The company is headquartered in San Francisco and is backed by venture capital and angel investors.


Model hallucinates, one of the ways of fixing it:

In [14]:
prompt = "Give me details about the technology startup called Mimi and Pimo. Only answer if you are 100% sure that this company exists, otherwise specify, 'I don't know '"

In [15]:
response = openai.Completion.create(engine = 'text-davinci-003',
                                    prompt = prompt,
                                    max_tokens=256,
                                    temperature = 0.7)

In [16]:
print(response['choices'][0]['text'])



I don't know.


## Creating our test prompt

In [32]:
def create_test_prompt(topic,num_questions,num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions."\
            +f" Each question should have {num_possible_answers} options."\
            +f" Also include the correct answer for each question using the starting string 'Correct answer:' "
    return prompt

In [21]:
create_test_prompt("Python",4,4)

"Create a multiple choice quiz on the topic of Python consisting of 4 questions. Each question should have 4 options. Also include the correct answer for each question using the starting string 'Correct answer' "

In [33]:
response = openai.Completion.create(engine = 'text-davinci-003',
                                    prompt = create_test_prompt('Spotify',4,4),
                                    max_tokens=256,
                                    temperature = 0.7)

In [34]:
print(response['choices'][0]['text'])



1. What is the name of the premium music streaming service offered by Spotify?
a. Prime Music
b. Spotify Premium
c. Music Plus
d. Music Now
Correct answer: b. Spotify Premium

2. How much does Spotify Premium cost?
a. $19.99
b. $9.99 
c. $14.99
d. $24.99
Correct answer: b. $9.99

3. What is the maximum number of devices that can be connected to one Spotify Premium account?
a. 4 devices
b. 5 devices
c. 6 devices
d. 7 devices
Correct answer: a. 4 devices

4. What is the maximum audio quality that Spotify offers?
a. 320 kbit/s
b. 192 kbit/s
c. 128 kbit/s
d. 64 kbit/s
Correct answer: a. 320 kbit/s


## Q&A Extraction


In [47]:
def create_student_view(test,num_question):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct answer:"):
            student_view[question_number]+= line+'\n'
        else:
            if question_number <num_question:
                question_number += 1
                student_view[question_number] = ''
    return student_view

In [48]:
result = create_student_view(response['choices'][0]['text'],4)

In [49]:
for key in result:
    print(result[key])



1. What is the name of the premium music streaming service offered by Spotify?
a. Prime Music
b. Spotify Premium
c. Music Plus
d. Music Now


2. How much does Spotify Premium cost?
a. $19.99
b. $9.99 
c. $14.99
d. $24.99


3. What is the maximum number of devices that can be connected to one Spotify Premium account?
a. 4 devices
b. 5 devices
c. 6 devices
d. 7 devices


4. What is the maximum audio quality that Spotify offers?
a. 320 kbit/s
b. 192 kbit/s
c. 128 kbit/s
d. 64 kbit/s



In [50]:
def extract_answer(test,num_question):
    answers = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct answer:"):
            answers[question_number]+= line+'\n'
            if question_number <num_question:
                question_number += 1
                answers[question_number] = ''
    return answers

In [51]:
extract_answer(response['choices'][0]['text'],4)

{1: 'Correct answer: b. Spotify Premium\n',
 2: 'Correct answer: b. $9.99\n',
 3: 'Correct answer: a. 4 devices\n',
 4: 'Correct answer: a. 320 kbit/s\n'}